In [1]:
#  Import the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
from cv2 import contourArea


In [2]:
# Import the .bmp image
image = cv2.imread('2000_LST.bmp', cv2.IMREAD_COLOR)

In [3]:
# Convert the image to grayscale
greyscale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [4]:
# Next we need to threshold the image to get a binary image
# We will use the Otsu's method to automatically calculate the threshold value
# The threshold value is returned in the variable 'threshold'
# The thresholded image is returned in the variable 'threshold_image'
_, threshold_image = cv2.threshold(greyscale, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# Find the contours in the thresholded image
# The contours are returned in the variable 'contours'
# The hierarchy is returned in the variable 'hierarchy'
contours, hierarchy = cv2.findContours(threshold_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

In [5]:
# Define a function to map RGB values to temperature
def map_rgb_to_temperature(r, g, b):
    # Convert RGB values to range [0, 1]
    r /= 255.0
    g /= 255.0
    b /= 255.0

    # Define temperature range
    hot_temperature = 100.0
    cold_temperature = 0.0

    # Map RGB values to temperature based on color mapping
    temperature = (1 - r) * hot_temperature + (1 - g) * cold_temperature

    return temperature


# Define a function to map temperature to color
def map_temperature_to_color(temperature):
    hot_color = (0, 0, 255)      # Red
    cold_color = (0, 255, 0)     # Green
    moderate_color = (255, 165, 0)  # Orange

    # Map temperature to color based on temperature ranges
    if temperature >= 80.0:
        return hot_color
    elif temperature <= 40.0:
        return cold_color
    else:
        return moderate_color


# Define a function to calculate temperature from RGB values
def calculate_temperature(r, g, b):
    # Convert RGB values to range [0, 1]
    r /= 255.0
    g /= 255.0
    b /= 255.0

    # Define temperature range
    hot_temperature = 100.0
    cold_temperature = 0.0

    # Map RGB values to temperature based on color mapping
    temperature = (1 - r) * hot_temperature + (1 - g) * cold_temperature

    return temperature


# Define a function to calculate the average temperature of a contour
def calculate_average_temperature(contour, image):
    # Extract pixel RGB values within contour
    temperature_data = []
    for point in contour:
        # Get RGB color values
        pixel = image[point[0][1], point[0][0]]
        r, g, b = pixel[2], pixel[1], pixel[0]

        # Map RGB values to temperature using your specific color mapping scheme
        temperature = calculate_temperature(r, g, b)

        # Calculate temperature value (additional calculations may be required)
        temperature_data.append(temperature)

    # Calculate average temperature
    average_temperature = np.mean(temperature_data)

    return average_temperature


In [6]:
# Next we will create a temperature map
# We will iterate over each contour and calculate the average temperature of each contour
# We will then map the average temperature to a color and draw the contour on the temperature map
temperature_map = np.zeros_like(image, dtype=np.uint8)
for contour in contours:
    # Calculate average temperature of contour
    average_temperature = calculate_average_temperature(contour, image)

    # Map temperature to color
    color = map_temperature_to_color(average_temperature)

    # Draw contour on temperature map
    cv2.drawContours(temperature_map, [contour], -1, color, thickness=cv2.FILLED)

# Display the temperature map
cv2.imshow('Temperature Map', temperature_map)
cv2.namedWindow('Temperature Map', cv2.WINDOW_NORMAL)
cv2.waitKey(0)
cv2.destroyAllWindows()

qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/flame/.local/lib/python3.10/site-packages/cv2/qt/plugins"


In [ ]:
# Save the temperature map
cv2.imwrite('temperature_map.png', temperature_map)
